# Web scraping assignment - 3 Solutions

In [1]:
#Importing libraries reqiured
import selenium
import pandas as pd
import time
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
import requests
import re
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait

1. Write a python program which searches all the product under a particular product from www.amazon.in.
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
#Connect to web driver
driver = webdriver.Chrome("chromedriver.exe")

#defining URL
url = 'https://www.amazon.in/'

#Getting the website to driver
driver.get(url)
#When we run this line, automatically the webpage will be opened

time.sleep(2) #adding sleep time of 2 seconds

#Searching for headphones in the search bar 
#Extracting the elements from the Search bar
search_bar = driver.find_element_by_id('twotabsearchtextbox')
search_bar

#Entering 'headphones' in the search bar
search_bar.send_keys('guitar')

time.sleep(2) #adding sleep time of 2 seconds

#Extracting the element of search option
search_btn = driver.find_element_by_id('nav-search-submit-button')
search_btn

#clicking on the search button
search_btn.click()

2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [3]:
# Fetching URLs to open the pages
urls = []    # Empty list
for i in range(0,3):   # for loop to scrape 3 pages
    Page_urls=driver.find_elements_by_xpath("//a[@class='a-link-normal s-no-outline']")#collecting urls of all the laptop
    for i in Page_urls:
        urls.append(i.get_attribute('href'))
        
    #next button 
    nxt_btn=driver.find_element_by_xpath("//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']") 
    url=nxt_btn.get_attribute('href')
    driver.get(url)
    time.sleep(2)

In [4]:
len(urls)

178

In [5]:
#Making empty lists and scraping the required spots
Product_name = []
Brand_name= []
Ratings = []
No_Ratings = []
Price = []
Return = []
Expected_Delivery = []
Availability = []
Other_Details = []

#Start with for loop
for i in urls:
    driver.get(i)
    time.sleep(2)
    
    
    #Scraping data for product name
    try:
        prod=driver.find_element_by_xpath("//span[@id='productTitle']")
        Product_name.append(prod.text)
    except NoSuchElementException as e:
        Product_name.append("-")
        
        
    #Scraping data for brand name
    try:
        brand=driver.find_element_by_xpath("//div[@id='bylineInfo_feature_div']/div/a")
        Brand_name.append(brand.text)
    except NoSuchElementException as e:
        Brand_name.append("-")
        
        
     #Scraping data for Ratings
    try:
        rat=driver.find_element_by_xpath("//span[@id='acrPopover']")
        Ratings.append(rat.get_attribute("title"))   
    except NoSuchElementException as e:
        Ratings.append("-")
        
        
    #Scraping data for No of Ratings
    try:
        no_rat=driver.find_element_by_xpath("//a[@id='acrCustomerReviewLink']/span")
        No_Ratings.append(no_rat.text)
    except NoSuchElementException as e:
        No_Ratings.append("-")
        
        
    #Scraping data for Price
    try:
        pri=driver.find_element_by_xpath("//span[@id='priceblock_ourprice']")
        Price.append(pri.text)
    except NoSuchElementException as e:
        Price.append("-")
        
     #Scraping data for Return/Exchange
    try:
        ret=driver.find_element_by_xpath("//div[@data-name='RETURNS_POLICY']/span/div[2]/a")
        Return.append(ret.text)
    except NoSuchElementException as e:
        Return.append("-")
        
     
    #Scraping data for Expected_Delivary
    try:
        delivary=driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
        Expected_Delivery.append(delivary.text)
    except NoSuchElementException as e:
        Expected_Delivery.append("-")
        
        
    #Scraping data for Availability
    try:
        avai=driver.find_element_by_xpath("//div[@id='availability']/span")
        Availability.append(avai.text)
    except NoSuchElementException as e:
        Availability.append("-")
        
        
    #Scraping data for Other_Details
    try:
        details=driver.find_element_by_xpath("//ul[@class='a-unordered-list a-vertical a-spacing-mini']")
        Other_Details.append(details.text)
    except NoSuchElementException as e:
        Other_Details.append("-")
        
        
#DATA FRAMEING
Guitar=pd.DataFrame({})
Guitar['Name'] = Product_name
Guitar['Brand'] = Brand_name
Guitar['Rating'] = Ratings
Guitar['No of Ratings'] = No_Ratings
Guitar['Price'] = Price
Guitar['Return'] = Return
Guitar['Expected_Delivery'] = Expected_Delivery
Guitar['Availability'] = Availability
Guitar['Other_Details'] = Other_Details
Guitar['Urls'] = urls

In [6]:
#Printing the dataframe
Guitar

,Name,Brand,Rating,No of Ratings,Price,Return,Expected_Delivery,Availability,Other_Details,Urls
0,Kadence Frontier guitar with Online Guitar lea...,Visit the Kadence Store,4.0 out of 5 stars,"1,394 ratings",-,7 Days Replacement,"Sunday, April 3",,"★ Design - The High Gloss finish, Simple head ...",https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,Visit the Kadence Store,3.9 out of 5 stars,"1,175 ratings",-,7 Days Replacement,-,In stock.,QUALITY STRINGS ---The surface is coated with ...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Vault Traveller 34 Inch Acoustic Guitar With B...,Visit the VAULT Store,3.9 out of 5 stars,298 ratings,-,7 Days Replacement,-,In stock.,"Size: 34"" Steel stringed | Number of Strings: ...",https://www.amazon.in/gp/slredirect/picassoRed...
3,GUITAR BRO - COMBO (Black Acoustic Guitar for ...,Visit the Guitar Bro Store,3.9 out of 5 stars,35 ratings,-,7 Days Replacement,-,In stock.,GUITAR BRO +20 mins FREE VIDEO Demo - is a lea...,https://www.amazon.in/gp/slredirect/picassoRed...
4,"Juarez Acoustic Guitar, 38 Inch Curved Body Cu...",Visit the JUAREZ Store,4.0 out of 5 stars,"1,871 ratings",-,7 Days Replacement,-,In stock.,"Jumbo Design, 38 Inch Acoustic Steel String Gu...",https://www.amazon.in/Juarez-Acoustic-Guitar-C...
...,...,...,...,...,...,...,...,...,...,...
173,Pluto Acoustic Best Guitar for Beginners HW34-...,Brand: Pluto,-,-,-,7 Days Replacement,-,,Oldest Brand : Most popular brand in India for...,https://www.amazon.in/Pluto-Acoustic-Beginners...
174,Techblaze Digital Guitar Tuner with Three Pick...,Brand: TECHBLAZE,3.3 out of 5 stars,14 ratings,-,7 Days Replacement,-,In stock.,★ PRO DESIGN For Guitar and Ukulele: Standard ...,https://www.amazon.in/Techblaze-Calibration-Ac...
175,Givson Cambridge CB-WR 6-String Cutaway Right ...,Visit the GIVSON Store,3.9 out of 5 stars,18 ratings,-,7 Days Replacement,-,Only 1 left in stock.,"40"" Cut-A-Way Guitar\nMaterial : Rosewood fing...",https://www.amazon.in/Givson-Cambridge-CB-WR-6...
176,Yamaha FSX80C Semi Acoustic Cutaway Guitar wit...,Visit the YAMAHA Store,5.0 out of 5 stars,2 ratings,-,7 Days Replacement,-,In stock.,Yamaha FSX80C semi acoustic guitar With a padd...,https://www.amazon.in/Yamaha-FSX80C-Acoustic-C...


In [7]:
driver.close()

3. Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’

In [8]:
#connecting to the webdriver
driver = webdriver.Chrome("chromedriver.exe")

#Getting the webpage of mentioned url
url = "https://images.google.com/"

#Creating list of items to be searched
search_items = [ 'fruits','cars','Machine Learning','Guitar','Cakes']

#creating empty list
urls = []
data = []

#creating a for loop to search all items in the search_item list

for item in search_items:
    driver.get(url)
    time.sleep(4) #sleep time of 4 seconds
    
    #Finding webelement for search_bar
    search_bar = driver.find_element_by_tag_name("input")
    
    #Sendingkeys to get the keyword for search bar
    search_bar.send_keys(str(item))
    
    #Clicking on the search button
    search_btn = driver.find_element_by_xpath("//button[@class='Tg7LZd']").click()
    
    #scrolling down the webpage to get some more images
    for _ in range(10):
        driver.execute_script("window.scrollBy(0,50)")
    
        time.sleep(5)
    
        imgs = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
            if(source[0:4] == 'http'):
                img_url.append(source)
    
    for i in img_url[:10]:
        urls.append(i)
        
    for i in range(len(urls)):
        if i >= 50:
            break
            
        print("Downloading {0} of {1} images" .format(i, 50))
        response = requests.get(urls[i])

        file = open(r"D:\fliproboProject\images"+str(i)+".jpg", "wb")

        file.write(response.content)

In [9]:
driver.close()

4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [10]:
# Asking for user input
prod = input(" Enter the name of the mobile phone you want to search : ")

#Connect to web driver
driver = webdriver.Chrome("chromedriver.exe")

# Getting the webpage of mentioned url 
url = "https://www.flipkart.com/"
driver.get(url)
#When we run this line, automatically the webpage will be opened

time.sleep(2)

#Closing the login popup
close_popup = driver.find_element_by_xpath('/html/body/div[2]/div/div/button')

#To click on the close button
close_popup.click()

time.sleep(3)   #3 seconds wait time for each page

 Enter the name of the mobile phone you want to search : android


In [11]:
#giving input key word to search bar

search_bar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']//input")

 # Inputing keyword to search
search_bar.send_keys(prod)

srch_btn = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
srch_btn.click()
time.sleep(5)

In [12]:
# Fetching urls of phones coming on 1st page
flip_urls = []
urls = driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for url in urls:
    flip_urls.append(url.get_attribute("href"))

In [13]:
len(flip_urls)

24

In [14]:
flip_dict = {}
flip_dict["Brand"] = []
flip_dict["Smartphone"] = []
flip_dict["Colour"] = []
flip_dict["RAM"] = []
flip_dict["Storage(ROM)"] = []
flip_dict["Primary Camera"] = []
flip_dict["Secondary Camera"] = []
flip_dict["Display Size"] = []
flip_dict["Display Resolution"] = []
flip_dict["Processor"] = []
flip_dict["Processor Cores"] = []
flip_dict["Battery Capacity"] = []
flip_dict["Battery Type"] = []
flip_dict["Price"] = []
flip_dict["URL"] = []

In [15]:
# Scraping data from each url
for url in flip_urls:
    driver.get(url)    # Saving url                                                     
    print("Scraping URL = ", url)
    flip_dict['URL'].append(url)                                                          # Loading the webpage by url
    time.sleep(2)
    
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     # Button for expanding the specs
        read_more.click()
    except NoSuchElementException:
        print("Exception Occured. Moving to next page")
    
    try:
        brand = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      # Extracting Brand from xpath
        flip_dict["Brand"].append(brand.text.split()[0])
    except NoSuchElementException:
        flip_dict['Brand'].append('-')
        
    try:
        price = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      # Extracting Brand from xpath
        flip_dict['Price'].append(price.text)
    except NoSuchElementException:
        flip_dict['Price'].append('-')
        
    try:
        name = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')      # Extracting Brand from xpath
        flip_dict['Smartphone'].append(name.text)
    except NoSuchElementException:
        flip_dict['Smartphone'].append('-')
    
    try:
        color = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      # Extracting Name from xpath
        flip_dict['Colour'].append(color.text)
    except NoSuchElementException:
        flip_dict['Colour'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  # Extracting Ratings from xpath
        flip_dict['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        flip_dict['Display Size'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_res = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[2]/td[2]/ul/li')     # Extracting no. of Ratings from xpath
        flip_dict['Display Resolution'].append(disp_res.text)
    except NoSuchElementException:
        flip_dict['Display Resolution'].append('-')
    
    try:
        pro_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
        if pro_chk.text != "Processor Type" : raise NoSuchElementException
        processor = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')   # Extracting Price from xpath
        flip_dict['Processor'].append(processor.text)
    except NoSuchElementException:
        flip_dict['Processor'].append('-')
    
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[1]')
        if core_chk.text != "Processor Core" :
            core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
            if core_chk.text != "Processor Core" : 
                raise NoSuchElementException
            else :
                cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')
        else :
            cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[2]/ul/li')
        flip_dict['Processor Cores'].append(cores.text)
    except NoSuchElementException:
        flip_dict['Processor Cores'].append('-')
    
    try:
        rom = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')         # Extracting Expected Delivery from xpath
        flip_dict['Storage(ROM)'].append(rom.text)
    except NoSuchElementException:
        flip_dict['Storage(ROM)'].append('-')
    
    try:
        ram = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['RAM'].append(ram.text)
    except NoSuchElementException:
        flip_dict['RAM'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        pri_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        flip_dict['Primary Camera'].append(pri_cam.text)
    except NoSuchElementException:
        flip_dict['Primary Camera'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        cam_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if cam_chk != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        flip_dict['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        flip_dict['Secondary Camera'].append('-')
        
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
            bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Capacity'].append(bat_cap.text)
    except NoSuchElementException:
        flip_dict['Battery Capacity'].append('-')
    
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[2]/ul/li')
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[1]')
            if bat_chk.text != "Battery Type" : raise NoSuchElementException
            bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Type'].append(bat_typ.text)
    except NoSuchElementException:
        flip_dict['Battery Type'].append('-')

Scraping URL =  https://www.flipkart.com/realme-c11-2021-cool-blue-64-gb/p/itmbd856acb97c38?pid=MOBG5BCDBHPYMT5B&lid=LSTMOBG5BCDBHPYMT5B5SJIEJ&marketplace=FLIPKART&q=android&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=Search&iid=176a51e0-e95b-49f8-a3c8-f1e431314996.MOBG5BCDBHPYMT5B.SEARCH&ppt=hp&ppn=homepage&ssid=00jg9vq61c0000001648632224067&qH=c31b32364ce19ca8
Scraping URL =  https://www.flipkart.com/realme-c11-2021-cool-grey-64-gb/p/itmbd856acb97c38?pid=MOBG5BCDJ68N57TH&lid=LSTMOBG5BCDJ68N57THWJHNPW&marketplace=FLIPKART&q=android&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=Search&iid=176a51e0-e95b-49f8-a3c8-f1e431314996.MOBG5BCDJ68N57TH.SEARCH&ppt=hp&ppn=homepage&ssid=00jg9vq61c0000001648632224067&qH=c31b32364ce19ca8
Scraping URL =  https://www.flipkart.com/redmi-9i-sport-carbon-black-64-gb/p/itm17ed463dc779a?pid=MOBG6WQWRGRRDBH6&lid=LSTMOBG6WQWRGRRDBH6DJKEOT&marketplace=FLIPKART&q=android&store=tyy%2F4io&srno=s_1_3&otracker=search&otracker1=sea

Scraping URL =  https://www.flipkart.com/redmi-10-midnight-black-128-gb/p/itm68b3d276eda8c?pid=MOBGC9GYX2QQXKWK&lid=LSTMOBGC9GYX2QQXKWKNFGDX0&marketplace=FLIPKART&q=android&store=tyy%2F4io&srno=s_1_23&otracker=search&otracker1=search&fm=Search&iid=176a51e0-e95b-49f8-a3c8-f1e431314996.MOBGC9GYX2QQXKWK.SEARCH&ppt=hp&ppn=homepage&ssid=00jg9vq61c0000001648632224067&qH=c31b32364ce19ca8
Scraping URL =  https://www.flipkart.com/kall-z4-grey-32-gb/p/itme838a62fef492?pid=MOBGA2E98APPDFJG&lid=LSTMOBGA2E98APPDFJGJSJ9RV&marketplace=FLIPKART&q=android&store=tyy%2F4io&srno=s_1_24&otracker=search&otracker1=search&fm=Search&iid=en_IO7qeniPm2%2F5EpWMtwLilLzcghp7sbS%2BJ2LXXSNLYyItTfPeHlib7MqA4rY21S63cYJ%2BsPMNjjeOdBvf9%2BF9cQ%3D%3D&ppt=hp&ppn=homepage&ssid=00jg9vq61c0000001648632224067&qH=c31b32364ce19ca8


In [16]:
#Checking lengths of all scraped data
print(len(flip_dict["Brand"]), len(flip_dict["Smartphone"]), len(flip_dict["Processor"]), len(flip_dict["Price"]), len(flip_dict['URL']))

24 24 24 24 24


In [17]:
flip_df = pd.DataFrame.from_dict(flip_dict)
flip_df

,Brand,Smartphone,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Battery Type,Price,URL
0,realme,C11 2021,Cool Blue,4 GB,64 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Octa-core,Octa Core,5000 mAh,-,"₹8,999",https://www.flipkart.com/realme-c11-2021-cool-...
1,realme,C11 2021,Cool Grey,4 GB,64 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Octa-core,Octa Core,5000 mAh,-,"₹8,999",https://www.flipkart.com/realme-c11-2021-cool-...
2,REDMI,9i Sport,Carbon Black,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,5000 mAh,-,"₹8,399",https://www.flipkart.com/redmi-9i-sport-carbon...
3,I,Z5,Blue,3 GB,16 GB,8MP Rear Camera,5MP Front Camera,13.84 cm (5.45 inch),480 x 960$$pixel,-,Quad Core,3000 mAh,-,"₹4,899",https://www.flipkart.com/kall-z5-blue-16-gb/p/...
4,REDMI,9i Sport,Coral Green,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,5000 mAh,-,"₹8,399",https://www.flipkart.com/redmi-9i-sport-coral-...
5,POCO,C31,Royal Blue,4 GB,64 GB,13MP + 2MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,5000 mAh,-,"₹8,999",https://www.flipkart.com/poco-c31-royal-blue-6...
6,realme,-,-,MediaTek Helio G35,Android 10,2 GB,-,-,-,-,-,5000 mAh,-,"₹7,499",https://www.flipkart.com/realme-c20-cool-blue-...
7,I,Z8,Cyan,3 GB,16 GB,8MP Rear Camera,5MP Front Camera,13.84 cm (5.45 inch),480 x 960$$pixel,-,Quad Core,2800 mAh,Lithium ion,"₹4,899",https://www.flipkart.com/kall-z8-cyan-16-gb/p/...
8,realme,-,-,MediaTek Helio G35,Android 10,2 GB,-,-,-,-,-,5000 mAh,-,"₹7,499",https://www.flipkart.com/realme-c20-cool-grey-...
9,realme,C25Y,Metal Grey,4 GB,64 GB,50MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Unisoc T610 Octa Core,Octa Core,5000 mAh,-,"₹10,890",https://www.flipkart.com/realme-c25y-metal-gre...


In [18]:
#saving data to csv
flip_df.to_csv('Flipkart_{}.csv'.format(item))

In [19]:
driver.close()

5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [20]:
# Connecting to the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# Getting mentioned url and opening google maps web page
url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(2)

# entering the city name in search bar 
City = input('Enter City name that has to be searched : ')
search_bar = driver.find_element_by_id("searchboxinput")                       
search_bar.clear()                                                             
time.sleep(2)

# Sending keys to find cities
search_bar.send_keys(City) 

# Checking for webelement and Clicking on search button
search_btn = driver.find_element_by_id("searchbox-searchbutton")              
search_btn.click()                                                             
time.sleep(2)

try:
    url_str = driver.current_url
    print("URL Extracted: ", url_str)
    latitude_longitude = re.findall(r'@(.*)data',url_str)
    if len(latitude_longitude):
        lat_lng_list = latitude_longitude[0].split(",")
        if len(lat_lng_list)>=2:
            latitude = lat_lng_list[0]
            longitude = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(latitude, longitude))

except Exception as e:
        print("Error: ", str(e))

Enter City name that has to be searched : Hyderabad
URL Extracted:  https://www.google.co.in/maps/place/Hyderabad,+Telangana/@17.4121531,78.1278451,10z/data=!3m1!4b1!4m5!3m4!1s0x3bcb99daeaebd2c7:0xae93b78392bafbc2!8m2!3d17.385044!4d78.486671
Latitude = 17.4121531, Longitude = 78.1278451


In [21]:
driver.close()

6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [22]:
# Connecting to the web driver
driver=webdriver.Chrome("chromedriver.exe") 

# opening the url track.in
url = "https://trak.in/"
driver.get(url)

time.sleep(2)

In [23]:
#Getting xpath for funding deals and clicking on the button
fundingdeals_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(fundingdeals_button)

In [24]:
# Empty lists
fund_deals = {}
fund_deals['Date'] = []
fund_deals['Startup Name'] = []
fund_deals['Industry/Vertical'] = []
fund_deals['Sub-Vertical'] = []
fund_deals['Location'] = []
fund_deals['Investor'] = []
fund_deals['Investment Type'] = []
fund_deals['Amount(in USD)'] = []

In [25]:
for i in range(54,57):

    # Scraping data of Date
    date = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in date:
        fund_deals['Date'].append(d.text)
        
fund_deals['Date']

['15/01/2021',
 '28/01/2021',
 '19/01/2021',
 '19/01/2021',
 '18/01/2021',
 '18/01/2021',
 '11/01/2021',
 '13/01/2021',
 '11/02/2021',
 '22/02/2021',
 '19/02/2021',
 '17/02/2021',
 '15/02/2021',
 '12/02/2021',
 '12/02/2021',
 '09/02/2021',
 '09/02/2021',
 '04/03/2021',
 '31/03/2021',
 '30/03/2021',
 '30/03/2021',
 '23/03/2021',
 '25/03/2021',
 '26/03/2021']

In [26]:
for i in range(54,57):
    # Scraping data of Startup Name
    startup_name= driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for name in startup_name:
        fund_deals['Startup Name'].append(name.text)
        
fund_deals['Startup Name']

['Digit Insurance',
 'Bombay Shaving Company',
 'DeHaat',
 'Darwinbox',
 'mfine',
 'Udayy',
 'True Elements',
 'Saveo',
 'Doubtnut',
 'Zomato',
 'Fingerlix',
 'Zolve',
 'KreditBee',
 'Pepperfry',
 'Grofers',
 'Nothing',
 'SplashLearn',
 'DealShare',
 'Uniphore',
 'Dunzo',
 'BYJU’S',
 'SkilloVilla',
 'CityMall',
 'DotPe']

In [27]:
for i in range(54,57):
    # Scraping data of Industry OR Vertical
    industry = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for ind in industry:
        fund_deals['Industry/Vertical'].append(ind.text)
        
fund_deals['Industry/Vertical']

['Financial Services',
 'Consumer Goods Company',
 'AgriTech Startup',
 'SaaS',
 'Health Tech Startup',
 'EdTech',
 'Food Startup',
 'B2B E-commerce',
 'Edu Tech',
 'Hospitality',
 'Hospitality',
 'FinTech',
 'Finance',
 'E-commerce',
 'E-Commerce',
 'Technology',
 'EdTech',
 'E-commerce',
 'Technology',
 'E-commerce',
 'Edu-tech',
 'Edu-tech',
 'E-commerce',
 'FinTech']

In [28]:
for i in range(54,57):    
    # Scraping data of Sub-Vertical
    sub_vertical = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for sv in sub_vertical:
        fund_deals['Sub-Vertical'].append(sv.text)
        
fund_deals['Sub-Vertical']

['Insurance Services',
 'Shave care, beard care, and skincare products',
 'online marketplace for farm products and services',
 'HR Tech',
 'AI-powered telemedicine mobile app',
 'Online learning platform for kids in class 1-5',
 'Whole Food plant based Nashta',
 'Pharmacies',
 'E-Learning Platform',
 'Online Food Delivery Platform',
 'Semi-cooked food delivery app',
 'Global Neobank Venture',
 'Digital lending platform',
 'Multi-brand furniture brand',
 'Online supermarket',
 'Consumer Technology Venture',
 'Game-based learning programme',
 'Online shopping platform',
 'Conversational Service Automation (CSA)',
 'Hyper-local delivery app',
 'Online tutoring',
 'Career and job-oriented upskilling.',
 'Social ecommerce and online grocery platform',
 'Commerce and payments platform to offline enterprise businesses.']

In [29]:
for i in range(54,57):        
    # Scraping data of Location
    location = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for loc in location:
        fund_deals['Location'].append(loc.text)
        
fund_deals['Location']

['Bengaluru',
 'New Delhi',
 'Patna',
 'Mumbai',
 'Bengaluru',
 'Gurgaon',
 'Pune',
 'Bengaluru',
 'Gurgaon',
 'Gurgaon',
 'Mumbai',
 'Mumbai',
 'Bengaluru',
 'Mumbai',
 'Gurgaon',
 'London',
 'Gurgaon',
 'Jaipur, Rajasthan',
 'Palo Alto',
 'Bengaluru',
 'Bengaluru',
 'Bengaluru',
 'Gurgaon',
 'Gurgaon']

In [30]:
for i in range(54,57):        
    # Scraping data of Investor
    investor = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for invest in investor:
        fund_deals['Investor'].append(invest.text)
        
fund_deals['Investor']

['A91 Partners, Faering Capital, TVS Capital Funds',
 'Reckitt Benckiser',
 'Prosus Ventures',
 'Salesforce Ventures',
 'Heritas Capital Management',
 'Sequoia Capital',
 'SIDBI Venture Capital',
 'Matrix Partners India, RTP Global, others',
 'SIG Global, Sequoia Capital, WaterBridge Ventures and ON Mauritius',
 'Tiger Global, Kora',
 'Rhodium Trust, Accel Partners and Swiggy',
 'Accel Partners and Lightspeed Venture Partners',
 'Azim Premji’s PremjiInvest and South Korea’s Mirae Asset Venture',
 'InnoVen Capital',
 'SoftBank Vision Fund (SVF)',
 'GV',
 'Owl Ventures',
 'Innoven Capital',
 'Sorenson Capital Partners',
 'Krishtal Advisors Pte Ltd',
 'MC Global Edtech, B Capital, Baron, others',
 'Titan Capital, others',
 'Accel Partners',
 'PayU']

In [31]:
for i in range(54,57):        
    # Scraping data of Investment Type
    investment_type = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for invtype in investment_type:
        fund_deals['Investment Type'].append(invtype.text)
        
fund_deals['Investment Type']

['Venture',
 'Venture',
 'Series C',
 'Seed',
 'Venture Round',
 'Seed Funding',
 'Series',
 'Seed',
 'Series B',
 'Venture',
 'Series C',
 'Seed',
 'Series C',
 'Debt Financing',
 'Unspecified',
 'Series A',
 'Series C',
 'Debt Financing',
 'Series D',
 'Series E',
 'Series F',
 'Seed',
 'Series A',
 'Series A']

In [32]:
for i in range(54,57):        
    # Scraping data of Amount
    amount = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for amt in amount:
        fund_deals['Amount(in USD)'].append(amt.text)
        
fund_deals['Amount(in USD)']

['1,80,00,000',
 '6,172,258.50',
 '30,000,000',
 '15,000,000',
 '16,000,000',
 '15,000,000',
 '100,000,000',
 '4,000,000',
 '2,500,000',
 '250,000,000',
 '2,747,045.20',
 '1,50,00,000',
 '75,000,000',
 '4,773,958',
 '55,000,000',
 '15,000,000',
 '18,000,000',
 '250,000,000',
 '140,000,000',
 '8,000,000',
 '460,000,000',
 '300,000,000',
 '11,000,000',
 '27,500,000']

In [33]:
print(len(fund_deals['Date']),len(fund_deals['Startup Name']),len(fund_deals['Industry/Vertical']),len(fund_deals['Sub-Vertical']),len(fund_deals['Location']),len(fund_deals['Investor']),len(fund_deals['Investment Type']),len(fund_deals['Amount(in USD)']))

24 24 24 24 24 24 24 24


In [34]:
# Creating DataFrame for scraped data
fund_data = pd.DataFrame(fund_deals)
fund_data

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [35]:
driver.close()

7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [36]:
# Connecting to the web driver
driver=webdriver.Chrome("chromedriver.exe") 

# opening the url
url = "https://digit.in/"
driver.get(url)
time.sleep(2)

In [37]:
#searching for best laptop
best_gam_lap = driver.find_element_by_xpath("//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)

In [38]:
#Creating empty lists
lap_name = []
ope_sys = []
display = []
processor = []
memory = []
weight = []
dimensions = []
graph_proc = []
price = []

In [39]:
# Scraping the data of laptop names
name_tags = driver.find_elements_by_xpath("//table[@id='summtable']//tr//td[1]")
for name in name_tags:
    lap_name.append(name.text)
    
    
#Scraping the data of operating system
try:
    os_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[3]//td[3]")
    for os in os_tags:
        ope_sys.append(os.text)
except NoSuchElementException:
    pass


#Scraping data of display
try:
    disp_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[4]//td[3]")
    for disp in disp_tags:
        display.append(disp.text)
except NoSuchElementException:
    pass


#Scraping data of Processor
try:
    pro_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[5]//td[3]")
    for pro in pro_tags:
        processor.append(pro.text)
except NoSuchElementException:
    pass


#Scraping data of memory
try:
    memo_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[6]//td[3]")
    for memo in memo_tags:
        memory.append(memo.text)
except NoSuchElementException:
    pass


#Scraping data of weight
try:
    wgt_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[7]//td[3]")
    for wgt in wgt_tags:
        weight.append(wgt.text)
except NoSuchElementException:
    pass


#Scraping data of dimensions
try:
    dim_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[8]//td[3]")
    for dim in dim_tags:
        dimensions.append(dim.text)
except NoSuchElementException:
    pass


#Scraping data of Graph processor
try:
    gra_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[9]//td[3]")
    for gra in gra_tags:
        graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#Scraping data of price
try:
    pri_tags = driver.find_elements_by_xpath("//td[@class='smprice']")
    for pri in pri_tags:
        price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass


In [40]:
#DATA FRAMEING
Gaming_Laptop=pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = lap_name
Gaming_Laptop['Operating system'] = ope_sys
Gaming_Laptop['Display'] = display
Gaming_Laptop['Processor'] = processor
Gaming_Laptop['Memory'] = memory
Gaming_Laptop['Weight'] = weight
Gaming_Laptop['Dimensions'] = dimensions
Gaming_Laptop['Graphical Processor'] = graph_proc
Gaming_Laptop['Price'] = price

In [41]:
#Printing data frame
Gaming_Laptop

,Laptop Name,Operating system,Display,Processor,Memory,Weight,Dimensions,Graphical Processor,Price
0,HP Victus 16,Windows 10,"16.1"" (1920 x 1200)",5th Gen AMD Ryzen 7 5800H | 3.2GHz,512 GB SSD/16 GBGB DDR4,2.48,37 x 26.2 x 2.4,NVIDIA GeForce RTX 3060,"Rs 104,990"
1,HP Omen 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen 7 Octa Core-5800H | 4.4 GHz,1 TB SSD/16 GBGB DDR4,2.37,357.9 x 239.7 x 22.5,NVIDIA GeForce RTX 3060,"Rs 110,790"
2,Acer Nitro 5,Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,1 TB HDD/16 GBGB DDR4,2.4,363.4 x 255 x 23.9,NVIDIA GeForce RTX 3070,"Rs 129,990"
3,MSI Stealth 15M,Windows 10,"15.6"" (1920 x 1080)",Intel Core i7 11th Gen - 11375H | NA,1 TB SSD/16 GBGB DDR4,1.7,358.3 x 248 x 16.15,NVIDIA GeForce RTX 3060,"Rs 134,990"
4,ASUS ROG Strix Scar 15,Windows 10,"15.6"" (2560 x 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,2 TB SSD/32 GBGB DDR4,2.30,354 x 259 x 22.6,NVIDIA GeForce RTX 3080,"Rs 193,990"
5,ASUS ROG Strix Scar 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,"Rs 215,990"
6,ASUS ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Gen Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"Rs 144,990"
7,Lenovo Legion 5i,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Gen Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,"Rs 76,988"
8,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"Rs 185,000"
9,Acer Aspire 7 gaming laptop,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,"Rs 53,490"


In [42]:
driver.close()

8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [43]:
# Connecting to the web driver
driver=webdriver.Chrome("chromedriver.exe") 

# opening the url
url = "https://www.forbes.com/?sh=69e6b8c92254"
driver.get(url)

#scrolling down the webpage to get some more images
for _ in range(100):
        driver.execute_script("window.scrollBy(0,50)")
    
time.sleep(5)

In [44]:
#clicking the explore button
button = driver.find_element_by_xpath("//button[@class='icon--hamburger']")
button.click()
time.sleep(1)

#select billionaire  
bill = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]")
bill.click()
time.sleep(1)

#select world billionaire  
world_bill= driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
world_bill.click()
time.sleep(1)

In [45]:
#Scraping required data from the web page
#Creating empty lists
Rank = []
Person_Name = []
total_net_worth = []
Age = []
citizenship = []
Source = []
industry = []


while(True):
    #Scraping the data of rank
    rank_tags= driver.find_elements_by_xpath("//div[@class='rank']")
    for rank in rank_tags:
        Rank.append(rank.text)
    time.sleep(1)
    
    
    #Scraping the data of names
    name_tags= driver.find_elements_by_xpath("//div[@class='personName']//div")
    for name in name_tags:
        Person_Name.append(name.text)
    time.sleep(1)
    
    
    #Scraping data of age
    age_tags= driver.find_elements_by_xpath("//div[@class='age']//div")
    for age in age_tags:
        Age.append(age.text)   
    time.sleep(1)
    
    
    #Scraping data of citizenship
    cit_tags= driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for cit in cit_tags:
        citizenship.append(cit.text)
    time.sleep(1)
    
    
    #Scraping data of source of income
    sour_tags= driver.find_elements_by_xpath("//div[@class='source']")
    for sour in sour_tags:
        Source.append(sour.text)    
    time.sleep(1)
    
    #Scraping data of Industry
    ind_tags= driver.find_elements_by_xpath("//div[@class='category']//div")
    for ind in ind_tags:
        industry.append(ind.text)
        
        
    #scraping data of net_worth of billionaire 
    net_tags= driver.find_elements_by_xpath("//div[@class='netWorth']//div[1]")
    for net in net_tags:
        total_net_worth.append(net.text)
    time.sleep(1)
    
    
    #Clicking on next button
    try:
        next_button = driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break
        
#Scraping data of net worth
Net_Worth = []
for i in range(0,len(total_net_worth),2):
    Net_Worth.append(total_net_worth[i])

In [46]:
#Framing data
Billionaires=pd.DataFrame({})
Billionaires['Rank'] = Rank
Billionaires['Names'] = Person_Name
Billionaires['Net Worth'] = Net_Worth
Billionaires['Age'] = Age
Billionaires['Citizenship'] = citizenship
Billionaires['Source'] = Source
Billionaires['Industry'] = industry

In [47]:
#Printing dataframe
Billionaires

,Rank,Names,Net Worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2750,2674.,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2751,2674.,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2752,2674.,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2753,2674.,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


In [48]:
driver.close()

9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [49]:
# Connecting to the web driver
driver=webdriver.Chrome("chromedriver.exe") 

# opening the url track.in
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(3)

In [50]:
#Extracting the element to close the ad
close_btn = driver.find_element_by_xpath('//*[@id="dismiss-button"]')

#closing the ad by clicking on close button
close_btn.click()

In [51]:
#Try sending the text using the ActionChains.
#You will need to import
from selenium.webdriver.common.action_chains import ActionChains

#Initialize the actions object with
actions = ActionChains(driver)

#Get the web element, let's call it "input"
input = driver.find_element_by_xpath('//input[@id="search"]')

#and then perform click and "send_keys" with the
actions.move_to_element(input).click().send_keys('Avengers').perform()

In [52]:
#clicking on search button
search_btn = driver.find_element_by_id("search-icon-legacy")  
search_btn.click()
time.sleep(1)

In [58]:
#clicking on first video
link_click = driver.find_element_by_xpath("//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link_click.click()

In [59]:
#scrolling down to generate more Comments
for _ in range(550):
    driver.execute_script("window.scrollBy(0,150)")

time.sleep(5)

In [60]:
#make empty lists
comments = []
Time = []
Likes = []

In [61]:
#scraping the data of comments
cm_tags = driver.find_elements_by_id("content-text")
for cm in cm_tags:
    if cm.text is None:
        comments.append("--")
    else:
        comments.append(cm.text)
time.sleep(5)

comments

['god i wish i could have the feeling of seeing this for the first time again',
 'One of the greatest moments in superhero history.',
 'Avengers 1: "I\'m always angry"\nAvengers 2: "I\'m often angry"\nAvengers 3: "I\'m sometimes angry"\nAvengers 4: "I\'m not angry"',
 '"Call it, Captain." \nThat\'s honestly one of the most underrated moments in the entire MCU.',
 'Even after all these years, Hulk punching the carrier still gives me chills.',
 'The smile on hulk\'s face when he heard " Smash " 👌',
 'one of the best scenes in MCU history',
 'cap’s leadership is so impressive that even hulk listens to him',
 '"That\'s my secret Captain. I\'m always angry." I think we can all agree That\'s one of the best lines from MCU',
 '2:22 this scene never gets old',
 "Ever found a scene so perfect that no matter how many years pass or how many more amazing scenes happen they still manage to give you the chills? Yes this is that scene it's almost 10 years now as of writing this comment but this scene

In [62]:
len(comments)

559

In [63]:
# scraping the data of time when comment was posted
tm_tags = driver.find_elements_by_xpath('//*[@id="header-author"]/yt-formatted-string/a')
for tm in tm_tags:
    Time.append(tm.text)

Time

['1 year ago',
 '3 months ago',
 '4 months ago',
 '2 months ago',
 '4 months ago',
 '2 months ago (edited)',
 '8 months ago',
 '3 months ago',
 '3 months ago',
 '2 months ago (edited)',
 '3 months ago',
 '2 months ago (edited)',
 '1 year ago',
 '3 months ago',
 '4 months ago',
 '1 month ago (edited)',
 '4 months ago',
 '6 days ago',
 '1 year ago',
 '1 month ago',
 '1 year ago (edited)',
 '2 weeks ago',
 '3 days ago',
 '4 days ago',
 '3 months ago',
 '1 month ago',
 '1 year ago',
 '1 month ago (edited)',
 '5 months ago',
 '9 days ago',
 '3 weeks ago',
 '4 days ago',
 '1 year ago',
 '3 months ago',
 '4 months ago',
 '2 months ago',
 '3 months ago',
 '1 month ago',
 '2 years ago',
 '3 months ago',
 '1 year ago',
 '5 months ago',
 '2 weeks ago',
 '2 months ago',
 '3 months ago',
 '1 month ago',
 '1 year ago',
 '3 months ago',
 '1 month ago',
 '3 weeks ago',
 '2 months ago',
 '2 months ago',
 '2 years ago',
 '3 months ago',
 '3 weeks ago (edited)',
 '2 months ago',
 '3 weeks ago',
 '1 month

In [64]:
len(Time)

559

In [65]:
# scraping the data of comment likes
like_tags = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
for like in like_tags:
    Likes.append(like.text)
    
Likes

['33K',
 '4.8K',
 '10K',
 '944',
 '3.4K',
 '166',
 '10K',
 '1.5K',
 '2.7K',
 '639',
 '427',
 '174',
 '19K',
 '155',
 '807',
 '57',
 '153',
 '15',
 '13K',
 '47',
 '12K',
 '14',
 '2',
 '1',
 '64',
 '19',
 '15K',
 '14',
 '36',
 '11',
 '13',
 '11',
 '5.8K',
 '64',
 '26',
 '135',
 '16',
 '7',
 '5.4K',
 '46',
 '6K',
 '120',
 '4',
 '14',
 '23',
 '1',
 '4.4K',
 '22',
 '4',
 '3',
 '14',
 '22',
 '2.1K',
 '25',
 '3',
 '14',
 '15',
 '3',
 '4.2K',
 '1',
 '1.4K',
 '2',
 '19',
 '5',
 '1',
 '1',
 '3.3K',
 '1',
 '1',
 '1',
 '4',
 '3',
 '2.7K',
 '2',
 '2',
 '2',
 '9',
 '1',
 '3.8K',
 '7',
 '3K',
 '6',
 '',
 '4',
 '2',
 '',
 '5.4K',
 '1',
 '4',
 '1',
 '3',
 '1',
 '4.1K',
 '5',
 '1',
 '6',
 '2',
 '',
 '2.1K',
 '1',
 '2.3K',
 '1',
 '5',
 '13',
 '4',
 '1',
 '5.5K',
 '1',
 '2',
 '',
 '8',
 '',
 '3.8K',
 '7',
 '',
 '1',
 '2',
 '1',
 '4.8K',
 '2',
 '5.4K',
 '1',
 '1',
 '1',
 '',
 '',
 '1.8K',
 '',
 '10',
 '1',
 '2',
 '1',
 '2.6K',
 '',
 '1',
 '1',
 '1',
 '1',
 '899',
 '',
 '3K',
 '',
 '1',
 '',
 '',
 '',
 '2.6

In [66]:
#Creating dataframe
Youtube=pd.DataFrame({})
Youtube['Comments'] = comments
Youtube['Comment_time'] = Time
Youtube['Comment upvotes'] = Likes

In [67]:
#Printing dataframe
Youtube[:500]

,Comments,Comment_time,Comment upvotes
0,god i wish i could have the feeling of seeing ...,1 year ago,33K
1,One of the greatest moments in superhero history.,3 months ago,4.8K
2,"Avengers 1: ""I'm always angry""\nAvengers 2: ""I...",4 months ago,10K
3,"""Call it, Captain."" \nThat's honestly one of t...",2 months ago,944
4,"Even after all these years, Hulk punching the ...",4 months ago,3.4K
...,...,...,...
495,it’s insane because no one understands (beside...,2 months ago,
496,this hulk wouldve clapped thanos💀,3 weeks ago,1
497,One of the most epic moments in cinema history,4 years ago,252
498,It feels strange to watch this in 60fps becaus...,2 months ago,


In [68]:
driver.close()

10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [118]:
# Connecting to the web driver
driver=webdriver.Chrome("chromedriver.exe") 

# opening the url
url = "https://www.hostelworld.com/"
driver.get(url)
time.sleep(2)

In [119]:
#Closing the login popup
close_banner = driver.find_element_by_xpath('//*[@id="__layout"]/div/div[1]/div[1]/div[2]')

#To click on the close button
close_banner.click()

In [120]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="search-input"]')))

#locating the location search bar
search_loc = driver.find_element_by_id("search-input-field")

# write Lonodn in search bar
search_loc.send_keys("London")
time.sleep(5)

#select london
london = driver.find_element_by_xpath('//*[@id="predicted-search-results"]/li[2]/div')
london.click()
time.sleep(5)


# To click on search button
search_btn = driver.find_element_by_xpath('//button[@class="button primary large"]')
search_btn.click()

In [121]:
#lets find required data
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description =[]
product_url = []

In [123]:
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@class ="pagination-item pagination-current"]')))

#Scraping the required informations
for i in driver.find_elements_by_xpath("//div[@class = 'pagination-item pagination-current' or @class='pagination-item']"):
    i.click()
    time.sleep(4)
    
    #Scraping hostel name
    try:
        name = driver.find_elements_by_xpath("//h2[@class='title title-6']")
        for i in name:
            hostel_name.append(i.text)
    except NoSuchElementException:
        hostel_name.append('-')
        
    #Scraping distance from city centre
    try:
        dist = driver.find_elements_by_xpath("//div[@class='subtitle body-3']//a//span[1]")
        for i in dist:
            distance.append(i.text.replace('Hostel - ',''))
    except NoSuchElementException:
        distance.append('-')
        
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
    #Scraping privates from price
        try:
            pvt_price = driver.find_element_by_xpath("//a[@class='prices']//div[1]//div")
            pvt_prices.append(pvt_price.text)
        except NoSuchElementException:
            pvt_prices.append('-')
            
    #Scraping dorms from price
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
        try:
            dorms = driver.find_element_by_xpath("//a[@class='prices']//div[2]//div")
            dorms_price.append(dorms.text)
        except NoSuchElementException:
            dorms_price.append('-')
            
    #Scraping facilities
    try:
        fac1 = driver.find_elements_by_xpath("//div[@class='has-wifi']")
        fac2 = driver.find_elements_by_xpath("//div[@class='has-sanitation']")
        for i in fac1:
            for j in fac2:
                facilities.append(i.text +', '+ j.text )
    except NoSuchElementException:
        facilities.append('-')
        
    #Fetching url of each hostel
    p_url = driver.find_elements_by_xpath("//div[@class='prices-col']//a[2]")
    for i in p_url:
        product_url.append(i.get_attribute('href'))

for i in product_url:
    driver.get(i)
    time.sleep(3)
    
    
    #Scraping ratings
    try:
        rat = driver.find_element_by_xpath("//div[@class='score orange big' or @class='score gray big']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
        
    #Scraping total reviews   
    try:
        rws = driver.find_element_by_xpath("//div[@class='reviews']")
        reviews.append(rws.text.replace('Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
        
    #Fetching overall review
    try:
        overall_rw = driver.find_element_by_xpath("//div[@class='keyword']//span")
        over_all.append(overall_rw.text)
    except NoSuchElementException:
        over_all.append('-')
        
    #Fetching property description 
    try:
        disc = driver.find_element_by_xpath("//div[@class='content']")
        description.append(disc.text)
    except NoSuchElementException:
        over_all.append('-')

In [124]:
print(len(hostel_name),len(facilities),len(distance),len(dorms_price),len(pvt_prices),len(product_url),len(rating),len(reviews),len(over_all),len(description))

58 764 58 58 58 58 58 58 58 58


In [126]:
#Creating dataframe
dff = pd.DataFrame({})
dff['Hostel_Name'] = hostel_name
dff['Distance fron city centre'] = distance
dff['Ratings'] = rating
dff['Total_reviews'] = reviews
dff['Overall Reviews'] = over_all
dff['Privates from price'] = pvt_prices
dff['Dorms from price'] = dorms_price
dff['Facilities'] = facilities[:58]
dff['Description'] = description

In [127]:
#Printing data frame
dff

,Hostel_Name,Distance fron city centre,Ratings,Total_reviews,Overall Reviews,Privates from price,Dorms from price,Facilities,Description
0,St Christopher's Village,1.8km from city centre,8.0,11065,Fabulous,Rs11896.78 Rs10707,Rs2926.41 Rs2634,"Free WiFi, Follows Covid-19 sanitation guidance","165 Borough High Street, London, England"
1,Safestay London Elephant & Castle,1.7km from city centre,7.7,4115,Very Good,Rs11896.78 Rs10707,Rs2926.41 Rs2634,"Free WiFi, Follows Covid-19 sanitation guidance","144-152 Walworth Road, London, England"
2,Generator London,3km from city centre,7.8,6847,Very Good,Rs11896.78 Rs10707,Rs2926.41 Rs2634,"Free WiFi, Follows Covid-19 sanitation guidance","Compton Place, Off 37 Tavistock Place, WC1, Lo..."
3,Urbany Hostel London,5.4km from city centre,9.6,225,Superb,Rs11896.78 Rs10707,Rs2926.41 Rs2634,"Free WiFi, Follows Covid-19 sanitation guidance","48-49 Princes Square, Paddington, London, England"
4,Smart Camden Inn Hostel,4.4km from city centre,8.5,2775,Fabulous,Rs11896.78 Rs10707,Rs2926.41 Rs2634,"Free WiFi, Follows Covid-19 sanitation guidance","55/57 Bayham Street, Camden, London, England"
5,London Backpackers,11.9km from city centre,8.2,4200,Fabulous,Rs11896.78 Rs10707,Rs2926.41 Rs2634,"Free WiFi, Follows Covid-19 sanitation guidance","8/10 Queens Parade, Queens Road, Hendon Centra..."
6,Smart Russell Square Hostel,2.6km from city centre,6.8,9593,Good,Rs11896.78 Rs10707,Rs2926.41 Rs2634,"Free WiFi, Follows Covid-19 sanitation guidance","71 Guilford Street, London, England"
7,YHA London Oxford Street,2.1km from city centre,9.6,4509,Superb,Rs11896.78 Rs10707,Rs2926.41 Rs2634,"Free WiFi, Follows Covid-19 sanitation guidance","14 Noel Street, Soho, W1F 8GJ, London, England"
8,Safestay London Kensington Holland Park,5.9km from city centre,7.7,1139,Very Good,Rs11896.78 Rs10707,Rs2926.41 Rs2634,"Free WiFi, Follows Covid-19 sanitation guidance","Holland Park Ave, Holland Walk, London, England"
9,Smart Hyde Park View Hostel,5km from city centre,7.9,4533,Very Good,Rs11896.78 Rs10707,Rs2926.41 Rs2634,"Free WiFi, Follows Covid-19 sanitation guidance","16 Leinster Terrace W2 3EU, London, England"


In [128]:
driver.close()